# Generating Models

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import datetime
import numpy as np
import matplotlib.pylab as plt
import sklearn
import sklearn.metrics
import itertools
import io

## Load Data

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

## Tensorboard stuff

In [3]:
def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.
    
    Args:
       cm (array, shape = [n, n]): a confusion matrix of integer classes
       class_names (array, shape = [n]): String names of the integer classes
    """
    
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    
    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)
    
    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)
        
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

def plot_to_image(figure):
    """
    Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call.
    """
    
    buf = io.BytesIO()
    
    # Use plt.savefig to save the plot to a PNG in memory.
    plt.savefig(buf, format='png')
    
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    
    # Use tf.image.decode_png to convert the PNG buffer
    # to a TF image. Make sure you use 4 channels.
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    
    # Use tf.expand_dims to add the batch dimension
    image = tf.expand_dims(image, 0)
    
    return image

def log_confusion_matrix(epoch, logs):
    
    # Use the model to predict the values from the test_images.
    test_pred_raw = model.predict(ds_test)
    
    test_pred = np.argmax(test_pred_raw, axis=1)
    
    test_labels = y = np.concatenate([y for x, y in ds_test], axis=0)
    
    # Calculate the confusion matrix using sklearn.metrics
    cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
    
    figure = plot_confusion_matrix(cm, class_names=[0,1,2,3,4,5,6,7,8,9])
    cm_image = plot_to_image(figure)
    
    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/model_one"
file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

## Model 1

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
    callbacks=[tensorboard_callback, cm_callback]
)

model.save("models/model_1")

Epoch 1/6
469/469 [==============================] - 6s 12ms/step - loss: 0.3967 - sparse_categorical_accuracy: 0.8874 - val_loss: 0.2976 - val_sparse_categorical_accuracy: 0.9134
Epoch 2/6
469/469 [==============================] - 6s 13ms/step - loss: 0.2921 - sparse_categorical_accuracy: 0.9170 - val_loss: 0.2824 - val_sparse_categorical_accuracy: 0.9210
Epoch 3/6
469/469 [==============================] - 4s 9ms/step - loss: 0.2782 - sparse_categorical_accuracy: 0.9226 - val_loss: 0.2729 - val_sparse_categorical_accuracy: 0.9236
Epoch 4/6
469/469 [==============================] - 4s 7ms/step - loss: 0.2710 - sparse_categorical_accuracy: 0.9250 - val_loss: 0.2852 - val_sparse_categorical_accuracy: 0.9193
Epoch 5/6
469/469 [==============================] - 4s 9ms/step - loss: 0.2677 - sparse_categorical_accuracy: 0.9256 - val_loss: 0.2716 - val_sparse_categorical_accuracy: 0.9260
Epoch 6/6
469/469 [==============================] - 5s 10ms/step - loss: 0.2622 - sparse_categorical_a

INFO:tensorflow:Assets written to: models/model_1/assets


## Model 2

In [6]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
    callbacks=[tensorboard_callback, cm_callback]
)

model.save("models/model_2")

Epoch 1/6
469/469 [==============================] - 6s 11ms/step - loss: 0.3527 - sparse_categorical_accuracy: 0.9028 - val_loss: 0.1976 - val_sparse_categorical_accuracy: 0.9414
Epoch 2/6
469/469 [==============================] - 4s 9ms/step - loss: 0.1670 - sparse_categorical_accuracy: 0.9523 - val_loss: 0.1416 - val_sparse_categorical_accuracy: 0.9574
Epoch 3/6
469/469 [==============================] - 5s 11ms/step - loss: 0.1198 - sparse_categorical_accuracy: 0.9659 - val_loss: 0.1133 - val_sparse_categorical_accuracy: 0.9662
Epoch 4/6
469/469 [==============================] - 4s 9ms/step - loss: 0.0936 - sparse_categorical_accuracy: 0.9728 - val_loss: 0.0976 - val_sparse_categorical_accuracy: 0.9714
Epoch 5/6
469/469 [==============================] - 4s 8ms/step - loss: 0.0751 - sparse_categorical_accuracy: 0.9787 - val_loss: 0.0888 - val_sparse_categorical_accuracy: 0.9750
Epoch 6/6
469/469 [==============================] - 5s 10ms/step - loss: 0.0613 - sparse_categorical_a

INFO:tensorflow:Assets written to: models/model_2/assets


## Model 3

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='sigmoid'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
    callbacks=[tensorboard_callback, cm_callback]
)

model.save("models/model_3")

Epoch 1/6
469/469 [==============================] - 7s 13ms/step - loss: 0.5922 - sparse_categorical_accuracy: 0.8602 - val_loss: 0.3077 - val_sparse_categorical_accuracy: 0.9161
Epoch 2/6
469/469 [==============================] - 4s 9ms/step - loss: 0.2775 - sparse_categorical_accuracy: 0.9223 - val_loss: 0.2365 - val_sparse_categorical_accuracy: 0.9321
Epoch 3/6
469/469 [==============================] - 4s 9ms/step - loss: 0.2220 - sparse_categorical_accuracy: 0.9368 - val_loss: 0.1984 - val_sparse_categorical_accuracy: 0.9424
Epoch 4/6
469/469 [==============================] - 4s 9ms/step - loss: 0.1851 - sparse_categorical_accuracy: 0.9471 - val_loss: 0.1701 - val_sparse_categorical_accuracy: 0.9505
Epoch 5/6
469/469 [==============================] - 4s 9ms/step - loss: 0.1586 - sparse_categorical_accuracy: 0.9546 - val_loss: 0.1505 - val_sparse_categorical_accuracy: 0.9560
Epoch 6/6
469/469 [==============================] - 5s 10ms/step - loss: 0.1374 - sparse_categorical_ac

INFO:tensorflow:Assets written to: models/model_3/assets


## Model 4

In [8]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation="tanh"),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
    callbacks=[tensorboard_callback, cm_callback]
)

model.save("models/model_4")

Epoch 1/6
469/469 [==============================] - 6s 10ms/step - loss: 0.3701 - sparse_categorical_accuracy: 0.8955 - val_loss: 0.2233 - val_sparse_categorical_accuracy: 0.9356
Epoch 2/6
469/469 [==============================] - 5s 9ms/step - loss: 0.1990 - sparse_categorical_accuracy: 0.9436 - val_loss: 0.1698 - val_sparse_categorical_accuracy: 0.9496
Epoch 3/6
469/469 [==============================] - 4s 9ms/step - loss: 0.1452 - sparse_categorical_accuracy: 0.9585 - val_loss: 0.1333 - val_sparse_categorical_accuracy: 0.9612
Epoch 4/6
469/469 [==============================] - 4s 9ms/step - loss: 0.1142 - sparse_categorical_accuracy: 0.9669 - val_loss: 0.1113 - val_sparse_categorical_accuracy: 0.9660
Epoch 5/6
469/469 [==============================] - 4s 9ms/step - loss: 0.0913 - sparse_categorical_accuracy: 0.9746 - val_loss: 0.0988 - val_sparse_categorical_accuracy: 0.9703
Epoch 6/6
469/469 [==============================] - 4s 9ms/step - loss: 0.0750 - sparse_categorical_acc

INFO:tensorflow:Assets written to: models/model_4/assets


## Model 5

In [13]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation="relu"),
  tf.keras.layers.Dense(64, activation="tanh"),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
    callbacks=[tensorboard_callback, cm_callback]
)

model.save("models/model_5")

Epoch 1/6
469/469 [==============================] - 7s 12ms/step - loss: 0.3112 - sparse_categorical_accuracy: 0.9122 - val_loss: 0.1504 - val_sparse_categorical_accuracy: 0.9541
Epoch 2/6
469/469 [==============================] - 4s 9ms/step - loss: 0.1252 - sparse_categorical_accuracy: 0.9625 - val_loss: 0.1112 - val_sparse_categorical_accuracy: 0.9657
Epoch 3/6
469/469 [==============================] - 4s 9ms/step - loss: 0.0854 - sparse_categorical_accuracy: 0.9742 - val_loss: 0.0916 - val_sparse_categorical_accuracy: 0.9720
Epoch 4/6
469/469 [==============================] - 5s 10ms/step - loss: 0.0636 - sparse_categorical_accuracy: 0.9804 - val_loss: 0.0763 - val_sparse_categorical_accuracy: 0.9759
Epoch 5/6
469/469 [==============================] - 4s 9ms/step - loss: 0.0487 - sparse_categorical_accuracy: 0.9854 - val_loss: 0.0769 - val_sparse_categorical_accuracy: 0.9747
Epoch 6/6
469/469 [==============================] - 4s 9ms/step - loss: 0.0385 - sparse_categorical_ac

INFO:tensorflow:Assets written to: models/model_5/assets
